# ISVR1032 Acoustics I
## String modes
### Matthew Wright mcmw@isvr.soton.ac.uk

In [1]:
%pylab
import matplotlib.animation as animation

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


## Overview
In the last notebook we used d'Alembert's solution of the one-dimensional wave equation to predict the motion of a plucked string. The result wasn't a perfect simulation of a real string because of the modelling assumptions that we made in order to make the problem simple enough to solve, which we can classify into:
- Wave equation assumptions (no stiffness, planar, small amplitude).
- Boundary conditions assumptions (rigidly fixed ends).
- Initial condition assumptions (stationary triangular shape).

Although the model was a deliberate simplification its predictions of the string's motion were good, at least for the first few periods. We were also able to predict the time-history of the force signal applied to the bridge. As noted it's unrealistic that this signal doesn't decay but there's another question that's of interest to Acoustical engineers: what does it sound like? What is its frequency content? We'll explore that question in this notebook.

We'll begin by taking the code we used to animate the plucked string in the previous notebook and use it to make a function `string_init()` that takes $y_\infty(\cdot)$ as an argument (i.e. an input) and produces the corresponding animation. We'll make the wave speed $c$ an argument too, but we'll leave the string length as a global variable because it will affect the form of $y_\infty(\cdot)$.

As before we'll display both the components and the actual string with a vertical offset, but this time we'll calculate the offset from $y_\infty(\cdot)$. We'll animate a whole period.

In [2]:
def string_init(y_inf, c):

    xs = linspace(0, L, 500)      # String from 0 to L
    x = linspace(-L, 2*L, 1000)     # Extended domain
    offset = 2*y_inf(x).max()

    movie = []                             
    fig, ax = subplots()
    for t in linspace(0, 2*L/c, 100):           
        frame = ax.plot(xs, (y_inf(xs - c*t) + y_inf(xs + c*t))/2 + offset, 'm',               # Actual string       
                        x, y_inf(x - c*t)/2, 'r',               # Right-going wave
                        x, y_inf(x + c*t)/2, 'b',               # Left-going wave
                        [0, 0, L, L], [0, offset, 0, offset], 'ro') # Dots
        ylim((-2*offset, 4*offset)) # Adjust the y-axis
        movie.append(frame)                 

    ani = animation.ArtistAnimation(fig, movie, interval=50, blit=True)     
    fig.show() 
    return ani

We need to return the resulting animation object `ani` because if we don't Python will assume we don't want it and won't produce an animation. For the same reason we'll need to store the output when we call our `string_init()` function.

Let's check it by calling it with the same plucked initial condition as before. This time we'll name the functions so as to indicate that they refer to this particular initial condition. 

In [3]:
A = 0.01
L = 0.64
beta = 0.3

def y_pluck(x):
    return (A/L)*((x/beta)*(x >= 0)*(x < beta*L) + (L - x)/(1 - beta)*(x >= beta*L)*(x <= L))

def y_inf_pluck_scalar(x):
    x = mod(x, 2*L)
    if 0 <= x <= L:
        return y_pluck(x)
    elif L < x <= 2*L:
        return -y_pluck(2*L - x)
    
y_inf_pluck = vectorize(y_inf_pluck_scalar)

Now we can call our function, specifying the wave speed $c$ and remembering to reference the returned animation object.

In [4]:
outer_ani = string_init(y_inf_pluck, 250)

#### Exercise
Try calling this with different values of wave speed $c$. What do you notice and how do you explain it?

## Sinusoidal initial conditions
If we had an initial condition

$$
y_0(x) = \sin\left(\frac{\pi x}{L}\right),
$$

it would satisfy the boundary conditions, because $\sin 0=0$ and $\sin\pi = 0$, and $y_\infty(x)$ would have a particularly simple form:

$$
y_\infty(x) = \sin\left(\frac{\pi x}{L}\right).
$$

We can use `string_init()` to show what the corrrsponding string motion will be.

In [5]:
def sinusoidal(x):
    return sin(pi*x/L)

outer_ani = string_init(sinusoidal, 200)

The equation for the string motion with this initial condition is

$$
\begin{align}
y(x,t) &= {\textstyle\frac{1}{2}} [y_\infty(x - ct) + y_\infty(x + ct)]\\
&= {\textstyle\frac{1}{2}} \sin\left[\frac{\pi(x - ct)}{L}\right] + {\textstyle\frac{1}{2}}\sin\left[\frac{\pi(x + ct)}{L}\right]
\end{align}
$$

The period of the motion is $T = 2L/c$ so its frequency is $f = 1/T = c/(2L)$ and its circular frequency is $\omega = 2\pi f = \pi c/L$ which we can use to write 

$$
y(x,t) = {\textstyle\frac{1}{2}} \sin\left(\frac{\pi x}{L} - \omega t\right) + {\textstyle\frac{1}{2}}\sin\left(\frac{\pi x}{L} + \omega t\right).
$$

If we take the sine addition/subtraction identities

$$
\begin{align}
\sin(A+B) &= \sin A \cos B + \sin B \cos A \\
\sin(A-B) &= \sin A \cos B - \sin B \cos A
\end{align}
$$

and add them we get

$$
\sin(A+B) + \sin(A-B) = 2\sin A \cos B,
$$

which, applied to $y(x,t)$, gives

$$
y(x,t) =  \sin\left(\frac{\pi x}{L}\right) \cos \omega t.
$$

This is an example of a **separable solution** of the wave equation, because the dependence on the two independent variables $x$ and $t$ is 'separated' into a product of two functions; one is a function of $x$ but not $t$, and the other is a function of $t$ but not $x$, i.e. 

$$
y(x,t) = p(x)\times q(t)
$$

Not all solutions of the wave equation are separable, for example the motion of our plucked string isn't separable. By inspecting the general form of the separable solution above we see that it must have the following properties:

1. ***When*** $q(t) = 0$ we must have $y(x,t) = 0\; \forall\, x$.
2. ***Where*** $p(x) = 0$ we must have $y(x,t) = 0\; \forall\, t$.
3. The shape of the string's displacement doesn't change with time except to grow and shrink in amplitude.

So if $y(x,t)$ is plotted as a surface then contour lines at $y(x,t)=0$ would be straight lines in the $x$ and $t$ directions, meeting at right angles.

The third property suggests that we don't have wave motion - this kind of motion caused by the superposition of two opposite-going waves of the same amplitude is called a **standing wave**.

#### Exercises
1. Run the animation of our sinusoidal initial condition again and observe the places where $y(x,t)= 0\;  \forall\, t $ and the times when  $y(x,t) = 0\; \forall\, x$.
2. Make a plot that verifies the statement above about the contours $y(x,t)=0$.

## Modes of vibration

The wave equation can always be separated into spatial and temporal factors with a sinusoidal temporal factor, whether there are one, two or three spatial dimensions. The spatial factors are called **modes of vibration** and every bounded continuous wave-bearing system has an infinite number of them.  When the waves are acoustic they're called This is also true of solid structure like plates, beams and car frames as long as they support linear waves, so you'll be hearing a lot more about modes as you study acoustical engineering.

We've found one mode of the string; what are the others? With only one spatial dimension they must fit the d'Alembert solution, which tells us how the temporal and spatial components must be related. If the time-variation is sinusoidal then the spatial variation must also be sinusoidal. Furthermore, if both boundaries reflect waves at their full amplitude and the initial velocity is zero, then the left and right-going components must be of equal amplitude, so by the derivation in the previous section they must be a scaled version of the first mode we found i.e.

$$
y(x,0) =  \sin\left(\frac{\alpha x}{L}\right) .
$$

This will satisfy the boundary condition at $x=0$ whatever $\alpha$ is, but it will only satisfy the condition at $x= L$ if $\alpha = \pi n$ where $n = 0,1,2,\ldots$, so

$$
y(x,0) =  \sin\left(\frac{\pi n x}{L}\right) .
$$


We can easily animate these modes if we define a function `mode()`. We'll make `n` a global variable so that `mode()` only takes a single argument, giving it the same syntax as `sinusoidal()`.

In [5]:
n = 5

def mode(x):
    return sin(n*pi*x/L)

outer_ani = string_init(mode, 200)

Try this with different values of $n$ and observe how the frequency of vbration changes, even though the component waves (red and blue) are always travelling at the same speed. We could work out how the frequency depends on $n$ by observing this animation, but we ought to be able to obtain it mathematically, so we'll try it that way. We have

$$
y(x,t) =  \sin\left(\frac{\pi nx}{L}\right) \cos \gamma t.
$$

where $\gamma$ is unknown, which must satisfy 

$$ 
\frac{\partial^2 y}{\partial x^2} - \frac{1}{c^2}\frac{\partial^2 y}{\partial t^2} = 0.
$$

We have

$$ 
\frac{\partial^2 y}{\partial x^2} = -\frac{\pi^2 n^2}{L^2}\sin\left(\frac{\pi nx}{L}\right)\cos \gamma t.
$$

and

$$ 
\frac{\partial^2 y}{\partial t^2} = -\gamma^2\sin\left(\frac{\pi nx}{L}\right)\cos \gamma t.
$$

so

$$
-\frac{\pi^2 n^2}{L^2} + \frac{\gamma^2}{c^2} = 0,
$$

which is satisfied when 

$$
\gamma = \frac{\pi n c}{L} = n \omega_1,
$$

where $\omega_1 = \pi c/L$ is the circular frequency of the first mode we found. So to summarize:
- Each mode has an associated frequency.
- The modal frequencies are all integer multiples of a fundamental frequency $f_1 = \omega_1/2\pi$.

In the next notebook we'll use these modes to predict the motion of the plucked string in a way that reveals the frequency content.

## Summary

[Edit this cell to summarise the important points from this notebook]